# 第３章

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ

In [1]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz

--2022-04-16 14:15:33--  https://nlp100.github.io/data/jawiki-country.json.gz
Resolving nlp100.github.io (nlp100.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5068362 (4.8M) [application/gzip]
Saving to: ‘jawiki-country.json.gz’

jawiki-country.json 100%[===================>]   4.83M  2.94MB/s    in 1.6s    

2022-04-16 14:15:36 (2.94 MB/s) - ‘jawiki-country.json.gz’ saved [5068362/5068362]



In [2]:
!gunzip jawiki-country.json.gz

gzip: jawiki-country.json already exists; do you wish to overwrite (y or n)? ^C


In [3]:
import json
with open('jawiki-country.json', 'r') as f:
    for l in f:
        data = json.loads(l)
        if data['title'] == 'イギリス':
            text = data['text']
            break

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [4]:
import re
re.findall(r'.*\[Category:.*', text)

['[[Category:イギリス|*]]',
 '[[Category:イギリス連邦加盟国]]',
 '[[Category:英連邦王国|*]]',
 '[[Category:G8加盟国]]',
 '[[Category:欧州連合加盟国|元]]',
 '[[Category:海洋国家]]',
 '[[Category:現存する君主国]]',
 '[[Category:島国]]',
 '[[Category:1801年に成立した国家・領域]]']

## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [5]:
re.findall(r'.*\[Category:(.+?)(?:\|.*)?\]', text)

['イギリス',
 'イギリス連邦加盟国',
 '英連邦王国',
 'G8加盟国',
 '欧州連合加盟国',
 '海洋国家',
 '現存する君主国',
 '島国',
 '1801年に成立した国家・領域']

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ

In [6]:
for l in re.findall(r'==.*==', text):
    print(l.replace('=', ''), l.count('=')//2-1)

国名 1
歴史 1
地理 1
主要都市 2
気候 2
政治 1
元首 2
法 2
内政 2
地方行政区分 2
外交・軍事 2
経済 1
鉱業 2
農業 2
貿易 2
不動産 2
エネルギー政策 2
通貨 2
企業 2
通信 3
交通 1
道路 2
鉄道 2
海運 2
航空 2
科学技術 1
国民 1
言語 2
宗教 2
婚姻 2
移住 2
教育 2
医療 2
文化 1
食文化 2
文学 2
哲学 2
音楽 2
ポピュラー音楽 3
映画 2
コメディ 2
国花 2
世界遺産 2
祝祭日 2
スポーツ 2
サッカー 3
クリケット 3
競馬 3
モータースポーツ 3
野球 3
 カーリング  3
 自転車競技  3
脚注 1
関連項目 1
外部リンク 1


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [7]:
re.findall(r'\[ファイル:(.*?)\|', text)

['Royal Coat of Arms of the United Kingdom.svg',
 'Descriptio Prime Tabulae Europae.jpg',
 "Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg",
 'London.bankofengland.arp.jpg',
 'Battle of Waterloo 1815.PNG',
 'Uk topo en.jpg',
 'BenNevis2005.jpg',
 'Population density UK 2011 census.png',
 '2019 Greenwich Peninsula & Canary Wharf.jpg',
 'Birmingham Skyline from Edgbaston Cricket Ground crop.jpg',
 'Leeds CBD at night.jpg',
 'Glasgow and the Clyde from the air (geograph 4665720).jpg',
 'Palace of Westminster, London - Feb 2007.jpg',
 'Scotland Parliament Holyrood.jpg',
 'Donald Trump and Theresa May (33998675310) (cropped).jpg',
 'Soldiers Trooping the Colour, 16th June 2007.jpg',
 'City of London skyline from London City Hall - Oct 2008.jpg',
 'Oil platform in the North SeaPros.jpg',
 'Eurostar at St Pancras Jan 2008.jpg',
 'Heathrow Terminal 5C Iwelumo-1.jpg',
 'Airbus A380-841 G-XLEB British Airways (10424102995).jpg',
 'UKpop.svg',
 'Anglospeak.svg',
 "Royal Aberdeen Children's Hospita

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [8]:
# 正規表現中の()はその部分だけを抽出する。
# A(?=B) 後方の文字列がBだった場合のAにだけ一致する
# A(?!B) 後方の文字列がBではない場合のAにだけ一致する
# (?:～) カッコ内の正規表現にマッチはするが拾ってはこない
# re.MULTILINE 文字列中の改行から、文中の行頭、行末も^, $でマッチするようにする
# re.DOTALL '.' を改行を含むあらゆる文字にマッチするようにする(本来'.'は改行以外のあらゆる文字にマッチする)
# .*?でなるべく少ない文字をマッチする

# ^{{基礎情報 『ここを拾う』 }}$
template = re.findall(r'^\{\{基礎情報.*?$(.+?)^\}\}$', text, re.MULTILINE + re.DOTALL)[0]
# ^| 『ここを拾う1』 = 『ここを拾う2』 |改行 or 改行$ までをワンセット
dic = dict(re.findall(r'^\|(.+?)\s*=\s*(.+?)(?:(?=\n\|)|(?=\n$))', template, re.MULTILINE + re.DOTALL))
dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save t

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ

他との区別（斜体） ''他との区別''
強調（太字） '''強調'''
斜体と強調 '''''斜体と強調'''''

In [9]:
dic = {key: re.sub(r'\'{2,}', '',val) for key, val in dic.items()}
dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save t

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ

[[記事名]]
[[記事名|表示文字]]
[[記事名#節名|表示文字]]

In [10]:
# [^パターン] パターンに一致しない1文字

# [[ 〜|〜|『ここを拾う』 ]]
dic = {key: re.sub(r'\[\[(?:[^|]*?\|)??([^|]*?)\]\]', r'\1',val) for key, val in dic.items()} 
dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:神と我が権利）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|ファイル:Unit

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ

In [11]:
# htmlタグ
dic = {key: re.sub(r'<.+?>', '',val) for key, val in dic.items()} 
# 外部リンク
dic ={key: re.sub(r'\[?http.+?(\||\])', '',val) for key, val in dic.items()} 
dic

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}英語以外での正式国名:\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}（フランス語:神と我が権利）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}神よ女王を護り賜え{{center|ファイル:United States Navy Band - God Save the 

In [12]:
# {{~|~|『ここを拾う』}}
dic = {key: re.sub(r'\*?\{\{(?:.*?\|)*?([^\|]+?)\}\}', r'\1',val, re.MULTILINE) for key, val in dic.items()} 
# ファイル:
dic = {key: re.sub(r'ファイル:([^\|]+?)', r'\1',val) for key, val in dic.items()} 
# [[『ここを拾う』|~]]
{key: re.sub(r'\[\[([^\|]+?)\|(.+?)\]\]', r'\1',val) for key, val in dic.items()} 
# その他
{key: re.sub(r'date\=2019-06-26', '',val) for key, val in dic.items()} 

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:\nAn Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）\nTeyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）\nRíocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）\nAn Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）\nUnitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）\n*Claught Kängrick o Docht Brätain an Norlin Airlann、Unitet Kängdom o Great Brittain an Norlin Airlann（アルスター・スコットランド語）',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（国章）',
 '標語': 'Dieu et mon droit（フランス語:神と我が権利）',
 '国歌': '[[女王陛下万歳|God Save the Queen]]en icon神よ女王を護り賜えUnited States Navy Band - God Save the Queen.ogg',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '英語',
 '首都': 'ロンド

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [13]:
import requests
url= dic['国旗画像'].replace(' ', '_')
url = 'https://commons.wikimedia.org/w/api.php?action=query&titles=File:' + url + '&prop=imageinfo&iiprop=url&format=json'
data = requests.get(url)
re.search(r'"url":"(.+?)"', data.text).group(1)

'https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg'